In [1]:
import numpy as np
from cellpose import models, core, io
from pathlib import Path
from pathlib import Path
import napari
import apoc
from tqdm import tqdm
import pyclesperanto_prototype as cle 
from tifffile import imwrite, imread
from utils import list_images, read_image

io.logger_setup() # run this to get printing of progress

#Check if colab notebook instance has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Load pre-trained Cellpose-SAM and Spotiflow models
model = models.CellposeModel(gpu=True)



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 


2025-10-06 12:32:45,429 [INFO] WRITING LOG OUTPUT TO C:\Users\adiez_cmic\.cellpose\run.log
2025-10-06 12:32:45,429 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0
2025-10-06 12:32:45,481 [INFO] ** TORCH CUDA version installed and working. **
2025-10-06 12:32:45,481 [INFO] ** TORCH CUDA version installed and working. **
2025-10-06 12:32:45,481 [INFO] >>>> using GPU (CUDA)
2025-10-06 12:32:46,434 [INFO] >>>> loading model C:\Users\adiez_cmic\.cellpose\models\cpsam


In [3]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path(r"\\forskning.it.ntnu.no\ntnu\mh\ikom\cmic_konfokal\lusie.f.kuraas\PhD\Nikon Spinning Disc\25.09.19\Plate 1 (occludin + claudin-2)")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

images

['\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E2_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E3_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E4_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E5_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E6_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E7_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie

In [4]:
viewer = napari.Viewer(ndisplay=2)

In [26]:
# Explore a different image (0 defines the first image in the directory)
image = images[0]

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)

# Extract well_id
well_id = filename.split("_")[0]

# Open one of the multipositions in the img file
img = img[2]

viewer.add_image(img)


Image analyzed: E2_20x


<Image layer 'img [1]' at 0x1e28ff01360>

In [28]:
img.shape

(15, 5, 1360, 1360)

In [30]:
img = img.transpose(1, 0, 2, 3)
img.shape

(5, 15, 1360, 1360)

In [35]:
# Cellmask and nuclei are in position 2 and 3 respectively

# if you want to combine two stains to create your "cytoplasm" channel
# in this example indices 0, 1 and 2 (1st, 2nd and 3rd) have cellular stains
# and nuclei are in index 3 (4th channel)

img_cp = np.stack((img[[0,1,2]].sum(axis=0), img[3]), axis=0)

img_cp.shape

(2, 15, 1360, 1360)

In [36]:
viewer.add_image(img_cp)

<Image layer 'img_cp [1]' at 0x1e216166830>

In [37]:
labels, flows, styles = model.eval(img_cp, channel_axis=0, z_axis=1, do_3D=True, niter=None) # need to check the arguments
viewer.add_labels(labels)

2025-10-06 13:20:07,504 [INFO] running YX: 15 planes of size (1360, 1360)
2025-10-06 13:20:20,161 [INFO] 100%|##########| 15/15 [00:12<00:00,  1.19it/s]
2025-10-06 13:20:20,311 [INFO] running ZY: 1360 planes of size (15, 1360)
2025-10-06 13:23:19,764 [INFO] 100%|##########| 1360/1360 [02:59<00:00,  7.58it/s]
2025-10-06 13:23:20,140 [INFO] running ZX: 1360 planes of size (15, 1360)
2025-10-06 13:26:10,267 [INFO] 100%|##########| 1360/1360 [02:50<00:00,  7.99it/s]
2025-10-06 13:26:10,916 [INFO] network run in 363.41s
2025-10-06 13:26:13,090 [INFO] masks created in 1.56s


<Labels layer 'labels' at 0x1e29ef268c0>

In [38]:
maxproj = np.max(img_cp, axis=1)

In [39]:
viewer.add_image(maxproj)

<Image layer 'maxproj' at 0x1e1f8deded0>

In [40]:
maxproj.shape

(2, 1360, 1360)

In [ ]:
labels, flows, styles = model.eval(maxproj[1], do_3D=False, niter=None) # simple nuclei segmentation
viewer.add_labels(labels)

<Labels layer 'labels [2]' at 0x1e298072290>

In [ ]:
cle.voronoi_otsu_labeling()

In [43]:
import nd2

with nd2.ND2File(image) as nd2_data:
    # Get the first channel's volume metadata
    first_channel = nd2_data.metadata.channels[0]
    voxel_size = first_channel.volume.axesCalibration  # X, Y, Z calibration

    # Extract pixel sizes
    pixel_size_x, pixel_size_y, voxel_size_z = voxel_size

    print(f"Pixel size: {pixel_size_x:.3f} µm x {pixel_size_y:.3f} µm")
    print(f"Voxel (Z-step) size: {voxel_size_z:.3f} µm")

Pixel size: 0.663 µm x 0.663 µm
Voxel (Z-step) size: 6.631 µm
